# 🎯 Problema de Negócio:

* A Vale S.A. (VALE3) é uma das principais empresas de mineração listadas na B3, com forte impacto no portfólio de investidores institucionais e pessoa física no Brasil. Para um gestor de investimentos ou analista de mercado, antecipar a trajetória do preço de fechamento (Close) e entender os padrões de volume de negociação é crucial para:

    1. Tomar decisões de compra/venda com melhor relação risco-retorno;

    2. Ajustar a alocação de ativos de curto a médio prazo;

    3. Desenvolver estratégias de trading baseadas em sinais estatísticos e de machine learning;

    4. Gerenciar o risco por meio de previsões confiáveis e métricas de incerteza.

* Objetivo Geral: Desenvolver um modelo preditivo de séries temporais para o preço de fechamento diário de VALE3, de modo a gerar forecasts de 7, 15 e 30 a dias futuros e embasar decisões de investimento.

* Importando as bibliotecas necessárias:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    r2_score
)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings("ignore")

# 1. Coletando e Tratando os dados.

In [2]:
# Coletando dados da VALE3 e formatando o DataFrame:
vale = yf.download('VALE3.SA', start='2022-01-01', end='2025-06-01', multi_level_index=False)
vale = vale[['Close', 'Open', 'Volume']]
vale.reset_index(inplace=True)
vale['Date'] = pd.to_datetime(vale['Date'])
vale.set_index('Date', inplace=True)

# Criando um DataFrame para armazenar os dados do minério de ferro:
minerio_ferro = pd.read_csv(
    "Dados Históricos - Minério de ferro refinado 62% Fe CFR Futuros.csv"
    )
minerio_ferro.reset_index(inplace=True)
minerio_ferro['Data'] = pd.to_datetime(minerio_ferro['Data'], dayfirst=True)
minerio_ferro.set_index('Data', inplace=True)
minerio_ferro = minerio_ferro[['Último','Abertura', 'Var%']]
# invertendo a ordem das datas:
minerio_ferro = minerio_ferro.iloc[::-1]
minerio_ferro.rename(columns={
    'Último': 'Close',
    'Abertura': 'Open',
    'Var%': 'Variação'
}, inplace=True)

# Unindo os DataFrames de VALE3 e minério de ferro:
df = pd.merge(vale, minerio_ferro, left_index=True, right_index=True, suffixes=('_VALE3', '_Minerio'))
df.rename(columns={
    'Close_VALE3': 'Close_VALE3',
    'Open_VALE3': 'Open_VALE3',
    'Volume': 'Volume_VALE3',
    'Close_Minerio': 'Close_Minerio',
    'Open_Minerio': 'Open_Minerio',
    'Variação': 'Variação_Minerio'
}, inplace=True)

# Variação percentual do preço de fechamento da VALE3:
df['Variação_VALE3'] = df['Close_VALE3'].pct_change() * 100
# Trocando nan por 0%:
df['Variação_VALE3'].fillna(0, inplace=True)

# Vamos transformar as colunas Close_Minerio, Open_Minerio e Variação_Minerio em float:
df['Close_Minerio'] = df['Close_Minerio'].str.replace(',', '.', regex=False).astype(float)
df['Open_Minerio'] = df['Open_Minerio'].str.replace(',', '.', regex=False).astype(float)
df['Variação_Minerio'] = df['Variação_Minerio'].str.replace('%', '', regex=False).str.replace(',', '.', regex=False).astype(float)

# Variação percentual do preço de fechamento do minério de ferro:
df['Variação_Minerio'] = df['Close_Minerio'].pct_change() * 100 
# Trocando nan por 7,02%:
df['Variação_Minerio'].fillna(7.02, inplace=True)

df

[*********************100%***********************]  1 of 1 completed


,Close_VALE3,Open_VALE3,Volume_VALE3,Close_Minerio,Open_Minerio,Variação_Minerio,Variação_VALE3
2022-01-03,57.766418,58.507014,18557200,120.40,120.40,7.020000,0.000000
2022-01-04,57.085068,58.144119,18178700,120.91,120.91,0.423588,-1.179493
2022-01-05,57.625694,57.299836,22039000,124.14,124.14,2.671408,0.947054
2022-01-06,58.788429,58.240391,22044100,125.94,125.94,1.449976,2.017737
2022-01-07,62.209984,59.543843,35213100,126.21,126.21,0.214388,5.820116
...,...,...,...,...,...,...,...
2025-05-23,54.320000,54.000000,14747300,99.81,99.81,-0.080088,0.165960
2025-05-27,53.840000,54.000000,17943000,99.48,99.48,-0.330628,-0.883652
2025-05-28,53.410000,53.740002,17700600,99.39,99.39,-0.090470,-0.798663
2025-05-29,53.450001,53.730000,10246100,99.27,99.27,-0.120736,0.074894


# 2. Adicionando fetures temporais ao nosso DataFrame.

In [3]:
# Vamos adicionar as fetures de dias, semanas, meses, anos e calcular o seno e o cosseno do mês para capturar sazonalidade cíclica:
df['Dia'] = df.index.day
df['Semana'] = df.index.weekday
df['Mês'] = df.index.month
df['Ano'] = df.index.year

df['Sin_Mês'] = np.sin(2 * np.pi * df['Mês'] / 12)
df['Cos_Mês'] = np.cos(2 * np.pi * df['Mês'] / 12)

# Vamos adicionar médias móveis de 20 e 200 dias no preço de fechamento da VALE3 e do minério de ferro:
df['MM_20D_VALE3'] = df['Close_VALE3'].rolling(window=20, min_periods=1).mean()
df['MM_200D_VALE3'] = df['Close_VALE3'].rolling(window=200, min_periods=1).mean()
df['MM_20D_MF'] = df['Close_Minerio'].rolling(window=20, min_periods=1).mean()
df['MM_200D_MF'] = df['Close_Minerio'].rolling(window=200, min_periods=1).mean()

# Coluna do preço anterior(D-1):
df['Close_VALE3_D-1'] = df['Close_VALE3'].shift(1)
df['Close_VALE3_D-1'].fillna(0, inplace=True)

df

,Close_VALE3,Open_VALE3,Volume_VALE3,Close_Minerio,Open_Minerio,Variação_Minerio,Variação_VALE3,Dia,Semana,Mês,Ano,Sin_Mês,Cos_Mês,MM_20D_VALE3,MM_200D_VALE3,MM_20D_MF,MM_200D_MF,Close_VALE3_D-1
2022-01-03,57.766418,58.507014,18557200,120.40,120.40,7.020000,0.000000,3,0,1,2022,0.5,0.866025,57.766418,57.766418,120.400000,120.400000,0.000000
2022-01-04,57.085068,58.144119,18178700,120.91,120.91,0.423588,-1.179493,4,1,1,2022,0.5,0.866025,57.425743,57.425743,120.655000,120.655000,57.766418
2022-01-05,57.625694,57.299836,22039000,124.14,124.14,2.671408,0.947054,5,2,1,2022,0.5,0.866025,57.492393,57.492393,121.816667,121.816667,57.085068
2022-01-06,58.788429,58.240391,22044100,125.94,125.94,1.449976,2.017737,6,3,1,2022,0.5,0.866025,57.816402,57.816402,122.847500,122.847500,57.625694
2022-01-07,62.209984,59.543843,35213100,126.21,126.21,0.214388,5.820116,7,4,1,2022,0.5,0.866025,58.695119,58.695119,123.520000,123.520000,58.788429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-23,54.320000,54.000000,14747300,99.81,99.81,-0.080088,0.165960,23,4,5,2025,0.5,-0.866025,54.055500,54.977746,99.505000,101.648300,54.230000
2025-05-27,53.840000,54.000000,17943000,99.48,99.48,-0.330628,-0.883652,27,1,5,2025,0.5,-0.866025,54.055000,54.964091,99.483000,101.611400,54.320000
2025-05-28,53.410000,53.740002,17700600,99.39,99.39,-0.090470,-0.798663,28,2,5,2025,0.5,-0.866025,54.023500,54.947779,99.457000,101.575400,53.840000
2025-05-29,53.450001,53.730000,10246100,99.27,99.27,-0.120736,0.074894,29,3,5,2025,0.5,-0.866025,54.004000,54.937926,99.427500,101.540500,53.410000


# 3. Decomposição da série temporal (tendência, sazonalidade e resíduo).

In [4]:
decomposicao = seasonal_decompose(df['Close_VALE3'], model='multiplicative', period=30)
df['Tendencia'] = decomposicao.trend
df['Sazonalidade'] = decomposicao.seasonal
df['Residuo'] = decomposicao.resid

# Preencher valores ausentes resultantes da decomposição:
df.fillna(method='bfill', inplace=True)
df.fillna(method='ffill', inplace=True)

# 4. Lags e Rolling Statistics.

* Vamos criar colunas que trazem o preço de fechamento e o retorno percentual de ontem, anteontem, etc. Assim o modelo “vê” vários dias anteriores:

In [5]:
lags = [1, 2, 3, 4, 5] # dias de atraso

for lag in lags:
    df[f'Close_Vale_lag{lag}'] = df['Close_VALE3'].shift(lag) # Preço do fechamento lagado
    df[f'Ret_Vale_lag{lag}'] = df['Close_VALE3'].shift(lag) # Retorno percentual lagado

* Agora, para captar tendências e volatilidade de curto e médio prazo, vamos criar médias móveis de curto prazo e estatísticas móveis (desvio, assimetria, curtose) em janelas de vários tamanhos:

In [6]:
windows = [5, 10, 15, 30, 50] # Janelas de dias

for w in windows:
    # Média móvel
    df[f'MM_de_{w}'] = df['Close_VALE3'].rolling(window=w, min_periods=1).mean()
    # Desvio-padrão móvel dos retornos em w dias
    df[f'Desvio_Padrao_de_{w}'] = df['Close_VALE3'].rolling(window=w, min_periods=1).std()
    # Assimetria móvel dos retornos
    df[f'roll_skew_{w}'] = df['Close_VALE3'].rolling(window=w, min_periods=1).skew()
    # Curtose móvel dos retornos
    df[f'Curtose_{w}'] = df['Close_VALE3'].rolling(window=w, min_periods=1).kurt()

In [7]:
# Dropando valores NaN:
df = df.dropna()
df

,Close_VALE3,Open_VALE3,Volume_VALE3,Close_Minerio,Open_Minerio,Variação_Minerio,Variação_VALE3,Dia,Semana,Mês,...,roll_skew_15,Curtose_15,MM_de_30,Desvio_Padrao_de_30,roll_skew_30,Curtose_30,MM_de_50,Desvio_Padrao_de_50,roll_skew_50,Curtose_50
2022-01-10,61.469395,61.091690,25056700,124.48,124.48,-1.370731,-1.190467,10,0,1,...,0.758624,-1.666765,59.157498,2.162186,0.758624,-1.666765,59.157498,2.162186,0.758624,-1.666765
2022-01-11,62.639538,61.824884,28418800,126.75,126.75,1.823586,1.903619,11,1,1,...,0.288821,-2.355135,59.654932,2.372332,0.288821,-2.355135,59.654932,2.372332,0.288821,-2.355135
2022-01-12,63.320869,64.054057,27335400,128.66,128.66,1.506903,1.087702,12,2,1,...,0.027979,-2.265001,60.113174,2.550265,0.027979,-2.265001,60.113174,2.550265,0.027979,-2.265001
2022-01-13,62.358105,62.832085,23154200,127.84,127.84,-0.637339,-1.520454,13,3,1,...,-0.244854,-2.134054,60.362611,2.500168,-0.244854,-2.134054,60.362611,2.500168,-0.244854,-2.134054
2022-01-14,62.721001,62.002623,21183400,126.24,126.24,-1.251564,0.581955,14,4,1,...,-0.457191,-1.914945,60.598450,2.472347,-0.457191,-1.914945,60.598450,2.472347,-0.457191,-1.914945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-23,54.320000,54.000000,14747300,99.81,99.81,-0.080088,0.165960,23,4,5,...,-0.324580,-1.637176,53.886667,1.031050,0.026033,-1.077892,54.707800,1.957700,-0.092500,-0.233223
2025-05-27,53.840000,54.000000,17943000,99.48,99.48,-0.330628,-0.883652,27,1,5,...,-0.414413,-1.341707,53.953000,0.956845,0.165655,-1.250139,54.709400,1.956942,-0.094601,-0.228681
2025-05-28,53.410000,53.740002,17700600,99.39,99.39,-0.090470,-0.798663,28,2,5,...,-0.421093,-1.216086,53.974000,0.936919,0.169704,-1.187451,54.687600,1.965375,-0.064842,-0.278782
2025-05-29,53.450001,53.730000,10246100,99.27,99.27,-0.120736,0.074894,29,3,5,...,-0.394918,-1.104977,53.967000,0.940125,0.186887,-1.207078,54.630800,1.959149,0.008174,-0.252883


# 5. Indicadores Técnicos.

* Média Móvel Exponencial:

In [8]:
# Média móvel exponencial de 20 dias:
df['MME_20D'] = df['Close_VALE3'].ewm(span=20, adjust=False).mean()
# Média móvel exponencial de 200 dias:
df['MME_200D'] = df['Close_VALE3'].ewm(span=200, adjust=False).mean()

* RSI (Relative Strength Index):

In [9]:
def calculo_rsi(series, window=14):
    delta = series.diff()

    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

df['RSI_14'] = calculo_rsi(df['Close_VALE3'], window=14)
df['RSI_14'].fillna(df['RSI_14'].mean(), inplace=True)
df[['RSI_14']]

,RSI_14
2022-01-10,49.585500
2022-01-11,49.585500
2022-01-12,49.585500
2022-01-13,49.585500
2022-01-14,49.585500
...,...
2025-05-23,63.709707
2025-05-27,57.777793
2025-05-28,54.450253
2025-05-29,56.327985


* MACD:

In [10]:
MME_12 = df['Close_VALE3'].ewm(span=12, adjust=False).mean()
MME_26 = df['Close_VALE3'].ewm(span=26, adjust=False).mean()

df['MACD'] = MME_12 - MME_26

df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

* Bollinger Bands:

In [12]:
# Calculando a média móvel e o desvio padrão
media_20 = df['Close_VALE3'].rolling(window=20, min_periods=1).mean()
desvio_20 = df['Close_VALE3'].rolling(window=20, min_periods=1).std()

# Calculando as bandas
df['Banda_Media'] = media_20
df['Banda_Superior'] = media_20 + (2 * desvio_20)
df['Banda_Inferior'] = media_20 - (2 * desvio_20)

df['Banda_Inferior'].fillna(df['Banda_Inferior'].mean(), inplace=True)
df['Banda_Superior'].fillna(df['Banda_Superior'].mean(), inplace=True)
df[['Banda_Media', 'Banda_Superior', 'Banda_Inferior']]

,Banda_Media,Banda_Superior,Banda_Inferior
2022-01-10,61.469395,63.620484,56.135745
2022-01-11,62.054466,63.709299,60.399634
2022-01-12,62.476601,64.349461,60.603741
2022-01-13,62.446977,63.980745,60.913209
2022-01-14,62.501781,63.852487,61.151076
...,...,...,...
2025-05-23,54.055500,56.080205,52.030795
2025-05-27,54.055000,56.079923,52.030077
2025-05-28,54.023500,56.068903,51.978097
2025-05-29,54.004000,56.064152,51.943848


# 6. Volatilidade e Risco.